In [ ]:
import astropy.units as u
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy.time import Time

import ipywidgets as ipw

import numpy as np

from stellarphot.differential_photometry.aij_rel_fluxes import *
from stellarphot.visualization.fits_opener import FitsOpener

### Load photometry table 

In [ ]:
fo = FitsOpener(title="Select your photometry file", filter_pattern=["*.csv"])
fo2 = FitsOpener(title="Select your aperture file", filter_pattern=["*.csv", "*.fits"])

In [ ]:
box = ipw.VBox()
box.children = [fo.file_chooser, fo2.file_chooser]
box

In [ ]:
aperture_file = fo2.path
photometry_file = fo.path

output_file = photometry_file.stem + "-relative-flux" + photometry_file.suffix

In [ ]:
photometry = Table.read(photometry_file)
del_rows = photometry['file'] == 'image_file.fits'
photometry = photometry[~del_rows]

### Load comp star table

In [ ]:
comp_table = Table.read(aperture_file)

In [ ]:
comp_table['coord'] = SkyCoord(ra=[c.split(',')[0] for c in comp_table['coord']],
                            dec=[c.split(',')[1] for c in comp_table['coord']],
                            unit='degree',
                            frame='icrs')

In [ ]:
comp_bool = comp_table['marker name'] == ['APASS comparison']
only_comp_stars = comp_table[comp_bool]
coords = only_comp_stars['coord']

In [ ]:
ra = [i.ra for i in coords]
dec = [i.dec for i in coords]
comp_coords = Table()
comp_coords['RA'] = ra * u.degree
comp_coords['Dec'] = dec * u.degree

### Calculate flux

In [ ]:
# calculate flux
flux_table = calc_aij_relative_flux(photometry, comp_coords)


In [ ]:
flux_table.write(output_file)